In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoporosis"
cohort = "GSE224330"

# Input paths
in_trait_dir = "../../input/GEO/Osteoporosis"
in_cohort_dir = "../../input/GEO/Osteoporosis/GSE224330"

# Output paths
out_data_file = "../../output/preprocess/Osteoporosis/GSE224330.csv"
out_gene_data_file = "../../output/preprocess/Osteoporosis/gene_data/GSE224330.csv"
out_clinical_data_file = "../../output/preprocess/Osteoporosis/clinical_data/GSE224330.csv"
json_path = "../../output/preprocess/Osteoporosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of monocytes from rheumatoid arthritis patients treated with bDMARDs and methotrexate."
!Series_summary	"It is well documented that patients affected by rheumatoid arthritis (RA) have distinct susceptibility to the different biologic Disease-Modifying AntiRheumatic Drugs (bDMARDs) available on the market, probably because of the many facets of the disease. Monocytes are deeply involved in the pathogenesis of RA and we therefore evaluated and compared the transcriptomic profile of monocytes isolated from patients on treatment with methotrexate alone or in combination with tocilizumab, anti-TNFalpha or abatacept, and from healthy donors. Differential expression analysis of whole-genome transcriptomics yielded a list of regulated genes suitable for functional annotation enrichment analysis. Specifically, abatacept, tocilizumab and anti-TNFalpha cohorts were separately compared with methotrexate using a rank-product-based statistical a

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# Based on the Series title and summary, this dataset does include gene expression data from monocytes
is_gene_available = True

# 2. Determine data availability and create conversion functions
# 2.1 Examining the sample characteristics dictionary
# Trait (Osteoporosis) availability
# Looking at key 3, it contains 'comorbidity' information including 'osteoporosis'
trait_row = 3

# Age availability (key 1 contains age data)
age_row = 1

# Gender availability (key 2 contains gender data)
gender_row = 2

# 2.2 Data type conversion functions
def convert_trait(value):
    """Convert trait value to binary (0 or 1)"""
    if pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates osteoporosis
    if 'osteoporosis' in value.lower():
        return 1
    elif 'none' in value.lower():
        return 0
    else:
        return 0  # Other comorbidities are not osteoporosis

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric age from strings like "63y"
    if 'y' in value:
        try:
            return int(value.replace('y', ''))
        except:
            return None
    try:
        return int(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    else:
        return None

# 3. Save metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only execute if trait_row is not None
if trait_row is not None:
    # Create clinical features dataframe
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Assumes clinical_data was defined in previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM7019507': [0.0, 63.0, 0.0], 'GSM7019508': [0.0, 64.0, 1.0], 'GSM7019509': [0.0, 63.0, 0.0], 'GSM7019510': [0.0, 48.0, 0.0], 'GSM7019511': [1.0, 70.0, 1.0], 'GSM7019512': [nan, 62.0, 1.0], 'GSM7019513': [nan, 58.0, 1.0], 'GSM7019514': [nan, 57.0, 1.0], 'GSM7019515': [nan, 60.0, 0.0], 'GSM7019516': [nan, 57.0, 0.0], 'GSM7019517': [nan, 52.0, 0.0], 'GSM7019518': [nan, 51.0, 0.0], 'GSM7019519': [nan, 53.0, 0.0], 'GSM7019520': [nan, 56.0, 0.0], 'GSM7019521': [nan, 62.0, 1.0], 'GSM7019522': [0.0, 54.0, 0.0], 'GSM7019523': [0.0, 61.0, 0.0], 'GSM7019524': [1.0, 54.0, 0.0], 'GSM7019525': [0.0, 55.0, 1.0], 'GSM7019526': [0.0, 65.0, 0.0], 'GSM7019527': [0.0, 84.0, 0.0], 'GSM7019528': [1.0, 70.0, 0.0], 'GSM7019529': [0.0, 76.0, 0.0], 'GSM7019530': [0.0, 62.0, 0.0], 'GSM7019531': [0.0, 73.0, 1.0], 'GSM7019532': [0.0, 71.0, 0.0], 'GSM7019533': [0.0, 59.0, 0.0], 'GSM7019534': [0.0, 62.0, 1.0], 'GSM7019535': [0.0, 47.0, 0.0], 'GSM7019536': [1.0, 76.0, 0.0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502',
       'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529',
       'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581',
       'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315625',
       'A_19_P00315627', 'A_19_P00315631', 'A_19_P00315641', 'A_19_P00315647'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (A_19_P00315452, etc.) are Agilent microarray probe IDs, not standard human gene symbols
# They follow the format A_19_P########, which is typical for Agilent technologies microarray probes
# These need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain identifiers and gene symbols
# Based on the preview, 'ID' contains the same kind of identifiers as in gene_data (Agilent probe IDs)
# and 'GENE_SYMBOL' contains the gene symbols we need

# 2. Get the gene mapping dataframe with these two columns
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Gene mapping dataframe shape: {mapping_data.shape}")
print("First few rows of mapping data:")
print(mapping_data.head())

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Save the gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (48862, 2)
First few rows of mapping data:
               ID    Gene
3   A_33_P3396872   CPED1
4   A_33_P3267760    BCOR
5    A_32_P194264   CHAC2
6    A_23_P153745   IFI30
10  A_21_P0014180  GPR146


Gene expression dataframe shape after mapping: (29222, 31)
First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1',
       'A2M-AS1', 'A2ML1', 'A2MP1'],
      dtype='object', name='Gene')


Gene data saved to ../../output/preprocess/Osteoporosis/gene_data/GSE224330.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from monocytes of rheumatoid arthritis patients, with osteoporosis status included in comorbidity information."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Osteoporosis/gene_data/GSE224330.csv
Clinical data saved to ../../output/preprocess/Osteoporosis/clinical_data/GSE224330.csv
Linked data shape: (31, 20781)


For the feature 'Osteoporosis', the least common label is '1.0' with 4 occurrences. This represents 19.05% of the dataset.
The distribution of the feature 'Osteoporosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 55.0
  50% (Median): 63.0
  75%: 70.0
Min: 47.0
Max: 84.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 5 occurrences. This represents 23.81% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Osteoporosis/GSE224330.csv
